## 

| 比较维度   | PyTorch                                           | Triton                                                              |
|------------|---------------------------------------------------|---------------------------------------------------------------------|
| 设计目的   | 深度学习框架，便于构建和训练神经网络             | 高性能计算和自定义 GPU 内核的优化                                  |
| 编程模型   | 高层抽象，易于使用                                 | 低级控制，专注于 GPU 内核开发                                      |
| 性能优势   | 依赖于底层库优化，适合大多数任务                 | 可编写高度优化的内核，实现更高性能                                  |
| 可扩展性   | 灵活但在低级优化方面依赖扩展工具                 | 提供更直接的 GPU 控制，支持自定义内核开发                          |
| 易用性     | 用户友好，适合新手和专家，入门门槛较低           | 需要 GPU 编程知识，适合有经验的用户                                 |
| 应用场景   | 适合标准深度学习任务，如图像分类和 NLP           | 适合需要高性能自定义计算和内核优化的场景                           |


### 1. 计算内核

创建一个辅助函数从而：

(1) 生成 `z` 张量

(2) 用适当的 grid/block sizes 将上述内核加入队列

In [1]:
import torch
import triton
import triton.language as tl

@triton.jit
# @triton.jit 装饰器：将此 Python 函数即时编译（Just-In-Time, JIT）为高性能的 GPU 可执行内核。
def add_kernel(
    x_ptr,         # 输入：指向第一个输入向量（x）在全局内存中起始地址的指针。
    y_ptr,         # 输入：指向第二个输入向量（y）在全局内存中起始地址的指针。
    output_ptr,    # 输出：指向输出向量在全局内存中起始地址的指针。
    n_elements,    # 输入参数：向量的元素总数，用于边界检查。
    BLOCK_SIZE: tl.constexpr,  # 编译时常量：定义单个程序实例处理的元素数量。
                               # 作为 `constexpr`，该值在编译阶段即被确定，
                               # 允许编译器执行特定优化，如循环展开和指令调度。
):
    """
    此 Triton 内核用于在 GPU 上高效执行两个向量的逐元素加法。
    其核心策略是将大规模计算任务分解为由多个并行程序实例处理的小数据块。
    """
    
    # 1. 程序标识与工作负载分配
    # GPU 以大规模并行方式启动内核，形成一个程序网格。
    # tl.program_id(axis=0) 用于获取当前程序实例在此一维网格中的唯一标识符（ID）。
    pid = tl.program_id(axis=0)

    # 基于程序ID（pid）和块大小（BLOCK_SIZE），计算当前程序实例负责处理的数据块的起始索引。
    # 确保不同的程序实例处理向量中不相交的数据段。
    block_start = pid * BLOCK_SIZE
    
    # tl.arange(0, BLOCK_SIZE) 生成一个 [0, 1, ..., BLOCK_SIZE-1] 的序列。
    # 与 block_start 相加后，得到当前程序实例需要操作的所有元素的绝对索引集合。
    # 此操作是向量化的，为后续的块级内存操作提供了基础。
    offsets = block_start + tl.arange(0, BLOCK_SIZE)

    # 2. 内存访问边界检查
    # 为保证程序的健壮性，必须防止内存访问越界，尤其是在向量总长度 n_elements
    # 不是 BLOCK_SIZE 的整数倍时，最后一个程序实例可能会访问无效内存。
    # 此处生成一个布尔类型的掩码，用于标识 offsets 中的索引是否在有效范围内。
    mask = offsets < n_elements

    # 3. 从全局内存 DRAM 加载数据
    # tl.load 指令执行向量化的块加载操作，将数据从慢速的 DRAM 传输到高速的 SRAM 或寄存器中。
    # `mask` 参数确保只加载掩码为真的有效数据，从而规避硬件异常。
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)

    # 4. 在片上内存中执行计算
    # 此时 x 和 y 均为驻留在高速缓存中的数据块。
    # 元素加法在此级别上执行，延迟极低，效率极高。
    output = x + y

    # 5. 将计算结果写回全局内存 DRAM
    # tl.store 指令将计算结果以向量化的块形式从片上内存写回到 DRAM。
    # 应用掩码 `mask` 可防止对向量边界之外的内存地址进行非法写入。
    tl.store(output_ptr + offsets, output, mask=mask)

ModuleNotFoundError: No module named 'triton'

定义一个使用 Triton 框架编写的自定义 GPU 内核函数，用于执行两个向量的逐元素加法运算，并将结果存储在输出向量中。

In [ ]:
def add(x: torch.Tensor, y: torch.Tensor):
    """
    此函数作为主机端的接口，负责调用 `add_kernel` Triton 内核来执行向量加法。
    它处理了内存分配、内核启动配置和内核执行的全过程。

    Args:
        x (torch.Tensor): 第一个输入张量，必须位于 CUDA 设备上。
        y (torch.Tensor): 第二个输入张量，必须位于 CUDA 设备上。

    Returns:
        torch.Tensor: 包含 x 和 y 逐元素相加结果的输出张量。
    """
    
    # 1. 输出内存预分配与验证
    # 在 CUDA 编程中，内核通常不负责内存分配。主机代码必须预先在 GPU 设备上
    # 为输出结果分配好内存空间。`torch.empty_like(x)` 创建一个与输入张量 x
    # 具有相同形状、数据类型和设备位置的未初始化张量。
    output = torch.empty_like(x)

    # 断言检查，确保所有涉及的张量都已存在于 CUDA 设备上。
    # Triton 内核只能操作 GPU 内存。
    assert x.is_cuda and y.is_cuda and output.is_cuda, "All tensors must be on a CUDA device."
    
    # 获取张量中的元素总数，该值将作为参数传递给内核，用于确定计算边界。
    n_elements = output.numel()

    # 2. 配置内核启动网格 
    # 启动网格定义了需要并行启动多少个内核实例。
    # 此处使用一维网格。网格的大小通过一个 lambda 函数动态计算：
    # `triton.cdiv(n_elements, meta['BLOCK_SIZE'])`
    # `triton.cdiv` 是向上取整除法，它确保即使 n_elements 不能被 BLOCK_SIZE 整除，
    # 也能启动足够数量的程序实例来覆盖所有元素。
    # `meta` 是一个字典，包含了在内核调用时传入的元参数（meta-parameters）。
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )

    # 3. 启动 Triton 内核
    # ====================
    # `add_kernel[grid]`：通过附加启动网格 `grid`，将 JIT 编译的 Triton 函数 `add_kernel`
    # 实例化为一个可调用的 GPU 内核对象。
    
    # `(...)`: 调用该内核对象，并传递参数。
    #  - 位置参数 (x, y, output, n_elements):
    #    - PyTorch 张量 `x`, `y`, `output` 会被自动转换成指向其数据在 GPU 内存中
    #      起始位置的指针。
    #    - `n_elements` 作为一个常规的整型值传递。
    #  - 关键字参数 (BLOCK_SIZE=1024):
    #    - 像 `BLOCK_SIZE` 这样的 `constexpr` 元参数必须作为关键字参数传递。
    #      这些值在内核编译时使用，会影响最终生成的代码。
    add_kernel[grid](x, y, output, n_elements, BLOCK_SIZE=1024)
    
    # 4. 返回结果句柄
    # GPU 操作本质上是异步的。函数返回的是 `output` 张量的句柄。
    # 对该张量的任何后续操作都会自动触发 CUDA 流的同步，等待内核执行完毕。
    return output